In [ ]:
# Text-to-Image Generation with Sunra AI

This notebook explores text-to-image generation in depth, covering various parameters, techniques, and best practices.

## What You'll Learn

- How to craft effective prompts
- Understanding model parameters
- Controlling image generation with seeds
- Different aspect ratios and formats
- Prompt enhancement techniques
- Batch generation and comparison


In [ ]:
## Setup

First, let's set up our environment and ensure we have the necessary libraries:


In [ ]:
import os
import sunra_client
import requests
from PIL import Image
import matplotlib.pyplot as plt
from io import BytesIO
import time
import getpass

# Set up API key if not already configured
if 'SUNRA_KEY' not in os.environ:
    api_key = getpass.getpass("Enter your Sunra API key: ")
    os.environ['SUNRA_KEY'] = api_key

# Configure the client
sunra_client.config(credentials=os.environ['SUNRA_KEY'])
print("✓ Sunra client configured successfully!")


In [ ]:
## Helper Functions

Let's create some helper functions to make our image generation and display easier:


In [ ]:
def display_image(image_url, title="Generated Image", size=(10, 10)):
    """Download and display an image from a URL"""
    try:
        response = requests.get(image_url)
        img = Image.open(BytesIO(response.content))
        
        plt.figure(figsize=size)
        plt.imshow(img)
        plt.axis('off')
        plt.title(title)
        plt.show()
        
        print(f"Image size: {img.size}")
        print(f"Image format: {img.format}")
        return img
        
    except Exception as e:
        print(f"Error displaying image: {e}")
        print(f"You can view the image directly at: {image_url}")
        return None

def generate_image(prompt, seed=None, aspect_ratio="1:1", output_format="jpeg", 
                   prompt_enhancer=False, safety_tolerance=6, show_image=True):
    """Generate an image with the given parameters"""
    
    print(f"Generating image with prompt: '{prompt}'")
    print("Please wait...")
    
    arguments = {
        "prompt": prompt,
        "prompt_enhancer": prompt_enhancer,
        "aspect_ratio": aspect_ratio,
        "output_format": output_format,
        "safety_tolerance": safety_tolerance
    }
    
    if seed is not None:
        arguments["seed"] = seed
    
    try:
        result = sunra_client.subscribe(
            "black-forest-labs/flux-kontext-pro/text-to-image",
            arguments=arguments,
            with_logs=True,
            on_enqueue=lambda req_id: print(f"✓ Request enqueued: {req_id}"),
            on_queue_update=lambda status: print(f"Status: {status}"),
        )

        if result.get('images'):
            image_url = result['images'][0]['url']
            print(f"✓ Image generated successfully!")
            print(f"Image URL: {image_url}")
            
            if show_image:
                display_image(image_url, f"Prompt: {prompt}")
            
            return result
        else:
            print("No image generated")
            return None
            
    except Exception as e:
        print(f"❌ Error generating image: {e}")
        return None

print("Helper functions ready!")


In [ ]:
# Simple image generation
result = generate_image("a cozy coffee shop with warm lighting and vintage decor")


In [ ]:
# Generate the same image twice with the same seed
prompt = "a majestic mountain landscape at sunset"
seed = 42

print("=== First generation with seed 42 ===")
result1 = generate_image(prompt, seed=seed)

print("\n=== Second generation with the same seed 42 ===")
result2 = generate_image(prompt, seed=seed)

print("\n=== Third generation with a different seed ===")
result3 = generate_image(prompt, seed=123)


In [ ]:
# Different aspect ratios
prompt = "a beautiful garden with flowers and butterflies"
seed = 100

aspect_ratios = ["1:1", "16:9", "9:16", "4:3", "3:4"]

for ratio in aspect_ratios:
    print(f"\n=== Aspect Ratio: {ratio} ===")
    result = generate_image(prompt, seed=seed, aspect_ratio=ratio, show_image=False)
    
    if result and result.get('images'):
        image_url = result['images'][0]['url']
        display_image(image_url, f"Aspect Ratio: {ratio}", size=(8, 6))


In [ ]:
# Compare with and without prompt enhancement
simple_prompt = "cat"
seed = 200

print("=== Without Prompt Enhancement ===")
result1 = generate_image(simple_prompt, seed=seed, prompt_enhancer=False)

print("\n=== With Prompt Enhancement ===")
result2 = generate_image(simple_prompt, seed=seed, prompt_enhancer=True)


In [ ]:
# Examples of different prompt styles
prompts = [
    "dog",  # Simple
    "a golden retriever sitting in a park",  # Descriptive
    "a golden retriever sitting in a park, soft lighting, professional photography, 4K",  # Detailed
    "a golden retriever sitting in a park, soft lighting, professional photography, 4K, realistic, highly detailed",  # Very detailed
]

seed = 300

for i, prompt in enumerate(prompts, 1):
    print(f"\n=== Prompt {i}: {prompt} ===")
    result = generate_image(prompt, seed=seed, show_image=False)
    
    if result and result.get('images'):
        image_url = result['images'][0]['url']
        display_image(image_url, f"Prompt {i}: {prompt}", size=(8, 6))


In [ ]:
def generate_variations(prompt, num_variations=3, base_seed=None):
    """Generate multiple variations of the same prompt"""
    
    if base_seed is None:
        base_seed = int(time.time()) % 1000  # Use current time as base seed
    
    print(f"Generating {num_variations} variations of: '{prompt}'")
    print(f"Base seed: {base_seed}")
    
    results = []
    
    for i in range(num_variations):
        seed = base_seed + i
        print(f"\n--- Variation {i+1} (seed: {seed}) ---")
        
        result = generate_image(prompt, seed=seed, show_image=False)
        
        if result and result.get('images'):
            image_url = result['images'][0]['url']
            display_image(image_url, f"Variation {i+1} - Seed: {seed}", size=(6, 6))
            results.append(result)
    
    return results

# Generate variations
variations = generate_variations("a futuristic city skyline at night", num_variations=3)


In [ ]:
# Advanced generation with different safety tolerance levels
prompt = "a beautiful sunset over mountains"
seed = 500

# Safety tolerance affects how strict the content filtering is
# Lower values = more strict, Higher values = more permissive
safety_levels = [3, 6, 9]

for level in safety_levels:
    print(f"\n=== Safety Tolerance: {level} ===")
    result = generate_image(
        prompt, 
        seed=seed, 
        safety_tolerance=level, 
        show_image=False
    )
    
    if result and result.get('images'):
        image_url = result['images'][0]['url']
        display_image(image_url, f"Safety Tolerance: {level}", size=(8, 6))


In [ ]:
# Creative and artistic prompts
creative_prompts = [
    "a steampunk octopus playing piano in an underwater jazz club",
    "a minimalist zen garden with floating geometric shapes",
    "a cyberpunk samurai meditating in a neon-lit temple",
    "a surreal melting clock forest in the style of Salvador Dali",
    "a cozy witch's cottage made of books and teacups"
]

for i, prompt in enumerate(creative_prompts, 1):
    print(f"\n=== Creative Example {i} ===")
    result = generate_image(prompt, seed=600+i, show_image=False)
    
    if result and result.get('images'):
        image_url = result['images'][0]['url']
        display_image(image_url, f"Creative #{i}", size=(8, 8))
